# Matchup Evaluator
This notebook is used to create code and logic that will help analyse an upcoming h2h Yahoo
Fantasy matchup

In [1]:
from src.data import all_functions as af
import pandas as pd

In [2]:
# Authenticate
sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)

[2021-02-06 09:51:38,366 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-06 09:51:38,380 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 38279.40076613426
[2021-02-06 09:51:38,381 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN HAS EXPIRED
[2021-02-06 09:51:38,381 DEBUG] [yahoo_oauth.oauth.refresh_access_token] REFRESHING TOKEN


In [8]:
# Nunn 9Cat averages
nunn_of_yall_betta_team = af.team_9cat_average_stats(af.NUNN_OF_YALL_BETTA, league)

In [9]:
nunn_of_yall_betta_team = nunn_of_yall_betta_team.loc["mean"]
nunn_of_yall_betta_team["PLAYER"] = af.NUNN_OF_YALL_BETTA["name"]
nunn_of_yall_betta_team = pd.DataFrame(nunn_of_yall_betta_team).T
nunn_of_yall_betta_team

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
mean,Nunn of Y'all Betta,0.462,0.817,1.95,15.17,4.99,3.68,0.99,0.37,1.67


In [11]:
# Crabbe 9Cat averages
crabbeherbythepussy_team = af.team_9cat_average_stats(af.CRABBEHERBYTHEPUSSY, league)

In [12]:
crabbeherbythepussy_team = crabbeherbythepussy_team.loc["mean"]
crabbeherbythepussy_team["PLAYER"] = af.CRABBEHERBYTHEPUSSY["name"]
crabbeherbythepussy_team = pd.DataFrame(crabbeherbythepussy_team).T
crabbeherbythepussy_team

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
mean,CrabbeHerByThePussy,0.486,0.817,1.52,14.22,6.17,3.02,0.88,0.92,1.63


In [18]:
# Drop PLAYER
nunn_of_yall_betta_team.drop("PLAYER", inplace=True, axis=1)
crabbeherbythepussy_team.drop("PLAYER", inplace=True, axis=1)

In [21]:
# Function to convert dataframe columns to dtype floats
def columns_to_dtype_float(dataframe):
    """
    Converts dataframe columns to dtype float.
    param dataframe (pandas dataframe): dataframe to change column types
    returns dataframe (pandas dataframe): dataframe with changed column types
    """
    for column in dataframe.columns:
        dataframe[column] = dataframe[column].astype(float)
    return dataframe

In [22]:
nunn_of_yall_betta_team = columns_to_dtype_float(nunn_of_yall_betta_team)
crabbeherbythepussy_team = columns_to_dtype_float(crabbeherbythepussy_team)

In [25]:
matchup_difference = nunn_of_yall_betta_team.sub(crabbeherbythepussy_team)
matchup_difference

,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
mean,-0.024,0.0,0.43,0.95,-1.18,0.66,0.11,-0.55,0.04


In [20]:
def naive_win_projection(dataframe):
    """

    """
    sum_of_9cat_differences = matchup_difference.sum(axis=1)

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, mean to mean
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   FG_PCT  1 non-null      object
 1   FT_PCT  1 non-null      object
 2   FG3M    1 non-null      object
 3   PTS     1 non-null      object
 4   REB     1 non-null      object
 5   AST     1 non-null      object
 6   STL     1 non-null      object
 7   BLK     1 non-null      object
 8   TOV     1 non-null      object
dtypes: object(9)
memory usage: 80.0+ bytes


In [30]:
matchup_difference.sum(axis=1)

mean    0.436
dtype: float64

In [ ]:
COLUMN_9CAT_DECIMAL_FORMAT = {af.FIELD_GOAL_PERCENTAGE_COLUMN : af.p9ca.THREE_DECIMAL_FORMAT,
     af.FREE_THROW_PERCENTAGE_COLUMN : af.p9ca.THREE_DECIMAL_FORMAT,
     af.THREES_MADE_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.POINTS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.REBOUNDS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.ASSITS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT, af.STEALS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.BLOCKS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT, af.TURNOVERS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT}

def applymap_color_and_format(dataframe):
    """

    """
    fromated_matchup_difference = matchup_difference.style.applymap(
        af.color_negative_red, subset=pd.IndexSlice[:,[af.FIELD_GOAL_PERCENTAGE_COLUMN,
                                                       af.FREE_THROW_PERCENTAGE_COLUMN,
                                                       af.THREES_MADE_COLUMN, af.POINTS_COLUMN,
                                                       af.REBOUNDS_COLUMN, af.ASSITS_COLUMN,
                                                       af.STEALS_COLUMN, af.BLOCKS_COLUMN
                                                       ]]).applymap(
        af.color_negative_red_tov, subset=pd.IndexSlice[:,[af.TURNOVERS_COLUMN]]).format(COLUMN_9CAT_DECIMAL_FORMAT)
    return fromated_matchup_difference

In [38]:
matchup_difference.style.applymap(
    af.color_negative_red, subset=pd.IndexSlice[:,["FG_PCT", "FT_PCT", "FG3M", "PTS", "REB", "AST",
                                                "STL", "BLK"]]).applymap(
    af.color_negative_red_tov, subset=pd.IndexSlice[:,["TOV"]]).format(COLUMN_9CAT_DECIMAL_FORMAT)

,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
mean,-0.024,0.000,0.43,0.95,-1.18,0.66,0.11,-0.55,0.04


In [4]:
PLAYER_COLUMN = "PLAYER"
MEAN_ROW = "mean"
NAME_KEY = "name"
COLUMN_9CAT_DECIMAL_FORMAT = {af.FIELD_GOAL_PERCENTAGE_COLUMN : af.p9ca.THREE_DECIMAL_FORMAT,
     af.FREE_THROW_PERCENTAGE_COLUMN : af.p9ca.THREE_DECIMAL_FORMAT,
     af.THREES_MADE_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.POINTS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.REBOUNDS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.ASSITS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT, af.STEALS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT,
     af.BLOCKS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT, af.TURNOVERS_COLUMN : af.p9ca.TWO_DECIMAL_FORMAT}

def columns_to_dtype_float(dataframe):
    """
    Converts dataframe columns to dtype float.
    param dataframe (pandas dataframe): dataframe to change column types
    returns dataframe (pandas dataframe): dataframe with changed column types
    """
    for column in dataframe.columns:
        dataframe[column] = dataframe[column].astype(float)
    return dataframe

def get_and_format_team_9cat(team, league):
    """
    Function first finds the teams 9cat averages by calling the *team_9cat_average_stats*
    function. It then filters to find the team mean and cleans and formats this single row
    dataframe. It changes the dtype of the columns to be floats.
    param team (dictionary): team dictionary
    param league (yahoo oauth object): yahoo authentication object
    returns team_9cat (dataframe): dataframe of team average 9cats
    """
    team_9cat = af.team_9cat_average_stats(team, league)
    team_9cat = team_9cat.loc[MEAN_ROW]
    team_9cat[PLAYER_COLUMN] = team[NAME_KEY]
    team_9cat = pd.DataFrame(team_9cat).T
    team_9cat.drop(PLAYER_COLUMN, inplace=True, axis=1)
    team_9cat = columns_to_dtype_float(team_9cat)
    return team_9cat

def applymap_color_and_format(dataframe):
    """
    Function takes a dataframe containing 9cat stat columns and returns a color coded and decimal
    place formatted dataframe for visual ease.
    param dataframe (pandas dataframe): dataframe to format
    returns dataframe (pandas dataframe): dataframe with formatted and colored values
    """
    colored_matchup_difference = dataframe.style.applymap(
        af.color_negative_red, subset=pd.IndexSlice[:,[af.FIELD_GOAL_PERCENTAGE_COLUMN,
                                                       af.FREE_THROW_PERCENTAGE_COLUMN,
                                                       af.THREES_MADE_COLUMN, af.POINTS_COLUMN,
                                                       af.REBOUNDS_COLUMN, af.ASSITS_COLUMN,
                                                       af.STEALS_COLUMN, af.BLOCKS_COLUMN
                                                       ]]).applymap(
        af.color_negative_red_tov, subset=pd.IndexSlice[:,[af.TURNOVERS_COLUMN]]).format\
        (COLUMN_9CAT_DECIMAL_FORMAT)
    return colored_matchup_difference

def compare_h2h_team_9cat(team1, team2):
    """
    Function authenticates on Yahoo and creates creates two dataframes containing team roster
    9cat averages for each team specified in the parameters. It calculates the difference between
    both 9cat averages and finally it formats the values for visual ease.
    params team1 (dictionary): team dictionary
    params team2 (dictionary): team dictionary
    return matchup_difference (dataframe): team difference 9cat dataframe
    return colored_matchup_difference (dataframe): formatted team difference 9cat dataframe
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team1_9cat_stats = get_and_format_team_9cat(team1, league)
    team2_9cat_stats = get_and_format_team_9cat(team2, league)
    matchup_difference = team1_9cat_stats.sub(team2_9cat_stats)
    colored_matchup_difference = applymap_color_and_format(matchup_difference)
    return matchup_difference, colored_matchup_difference

In [5]:
matchup_difference, colored_matchup_difference = compare_h2h_team_9cat(af.NUNN_OF_YALL_BETTA, af
                                                                       .CRABBEHERBYTHEPUSSY)

[2021-02-05 18:33:47,632 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-05 18:33:47,648 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 167.65970396995544
[2021-02-05 18:33:47,648 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [6]:
colored_matchup_difference


,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
mean,-0.028,-0.001,0.43,1.06,-1.21,0.66,0.12,-0.55,0.02


In [7]:
matchup_difference


,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
mean,-0.028,-0.001,0.43,1.06,-1.21,0.66,0.12,-0.55,0.02


In [17]:
positive_list = list()
negative_list = list()

for value in matchup_difference.values.tolist()[0]:
    if value > 0:
        positive_list.append(value)
    elif value < 0:
        negative_list.append(abs(value))

In [19]:
print(f"Sum of positive list is: {sum(positive_list)}")
print(f"Sum of negative list is: {sum(negative_list)}")

Sum of positive list is: 2.2900000000000005
Sum of negative list is: 1.789


In [3]:
# Authenticate
sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)


[2021-02-06 00:10:07,490 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-06 00:10:07,493 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3388.5129010677338
[2021-02-06 00:10:07,495 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


## Free Agent Sort by Rank
This section will work on logic that returns the Leaugue's FA sorted by Rank. This needs to be
done because it will be faster to calculate stats for the top 25 FA rather than all +400 in Yahoo

In [7]:
from yahoo_fantasy_api import yhandler
import yahoo_fantasy_api as yfa
yh = yhandler.YHandler(sc)
gm = yfa.Game(sc, 'nba')
# Get League Ids I have access to
league_id_list = gm.league_ids(year=2020)
league_id = "".join(str(id) for id in league_id_list)

In [9]:
yh.get(f"/league/{league_id}/players;status=FA;sort=AR")

{'fantasy_content': {'xml:lang': 'en-US',
  'yahoo:uri': '/fantasy/v2/league/402.l.55374/players;status=FA;sort=AR',
  'league': [{'league_key': '402.l.55374',
    'league_id': '55374',
    'name': 'Lebrontourage',
    'url': 'https://basketball.fantasysports.yahoo.com/nba/55374',
    'logo_url': 'https://yahoofantasysports-res.cloudinary.com/image/upload/t_s192sq/fantasy-logos/cab87bf0fa8d01c5ba29f76654390cf6fed3776b77431028f7a94f67e6d67976.jpg',
    'draft_status': 'postdraft',
    'num_teams': 12,
    'edit_key': '2021-02-05',
    'weekly_deadline': 'intraday',
    'league_update_timestamp': '1612510551',
    'scoring_type': 'headone',
    'league_type': 'private',
    'renew': '395_53528',
    'renewed': '',
    'iris_group_chat_id': '',
    'allow_add_to_dl_extra_pos': 0,
    'is_pro_league': '0',
    'is_cash_league': '0',
    'current_week': 7,
    'start_week': '1',
    'start_date': '2020-12-22',
    'end_week': '20',
    'end_date': '2021-05-16',
    'game_code': 'nba',
    '

## Get Week Date
This section will create logic needed to return the current week number (i.e. Week 5). This will
allow for the date, start and end, to be found.


In [3]:
# Authenticate
sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)


[2021-02-06 09:51:45,621 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-02-06 09:51:45,623 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 7.2402260303497314
[2021-02-06 09:51:45,624 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [6]:
# Current week
current_fantasy_week = league.current_week()

# Next week
next_fantasy_week = current_fantasy_week + 1
print(f"Next weeks fantasy mathcup will fall under week {next_fantasy_week}")

Next weeks fantasy mathcup will fall under week 8


In [11]:
# Get dates
week_start_date = league.week_date_range(current_fantasy_week)[0]

In [14]:
week_start_date.strftime("%A, %d %B %Y")

'Monday, 01 February 2021'

In [15]:
from nba_api.stats.endpoints import leaguegamefinder

In [42]:
leaguegamefinder.LeagueGameFinder(date_from_nullable='2021-01-03', date_to_nullable="2021-01-08",
                                  season_nullable = 2020)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [47]:
from nba_api.stats.endpoints import leaguegamelog
league_log = leaguegamelog.LeagueGameLog(
counter=1,
direction='ASC',
season = 2020,
sorter = 'DATE',
date_from_nullable = '2020-11-17',
date_to_nullable = '2021-04-17'
)

In [48]:
league_log

In [49]:
logs = league_log.league_game_log.get_data_frame()
print(logs)


    SEASON_ID     TEAM_ID TEAM_ABBREVIATION              TEAM_NAME  \
0       22020  1610612744               GSW  Golden State Warriors   
1       22020  1610612751               BKN          Brooklyn Nets   
2       22020  1610612747               LAL     Los Angeles Lakers   
3       22020  1610612746               LAC            LA Clippers   
4       22020  1610612739               CLE    Cleveland Cavaliers   
..        ...         ...               ...                    ...   
651     22020  1610612764               WAS     Washington Wizards   
652     22020  1610612765               DET        Detroit Pistons   
653     22020  1610612756               PHX           Phoenix Suns   
654     22020  1610612738               BOS         Boston Celtics   
655     22020  1610612746               LAC            LA Clippers   

        GAME_ID   GAME_DATE      MATCHUP WL  MIN  FGM  ...  DREB  REB  AST  \
0    0022000001  2020-12-22    GSW @ BKN  L  240   37  ...    34   47   26   
1  

In [50]:
logs

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22020,1610612744,GSW,Golden State Warriors,0022000001,2020-12-22,GSW @ BKN,L,240,37,...,34,47,26,6,6,18,24,99,-26,1
1,22020,1610612751,BKN,Brooklyn Nets,0022000001,2020-12-22,BKN vs. GSW,W,240,42,...,44,57,24,11,7,20,22,125,26,1
2,22020,1610612747,LAL,Los Angeles Lakers,0022000002,2020-12-22,LAL vs. LAC,L,240,38,...,37,45,22,4,2,19,20,109,-7,1
3,22020,1610612746,LAC,LA Clippers,0022000002,2020-12-22,LAC @ LAL,W,240,44,...,29,40,22,10,3,16,29,116,7,1
4,22020,1610612739,CLE,Cleveland Cavaliers,0022000010,2020-12-23,CLE vs. CHA,W,240,46,...,40,50,34,12,3,21,22,121,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,22020,1610612764,WAS,Washington Wizards,0022000347,2021-02-05,WAS @ MIA,L,240,33,...,28,40,19,7,9,18,20,95,-27,1
652,22020,1610612765,DET,Detroit Pistons,0022000349,2021-02-05,DET @ PHX,L,240,34,...,28,45,20,8,6,13,16,92,-17,1
653,22020,1610612756,PHX,Phoenix Suns,0022000349,2021-02-05,PHX vs. DET,W,240,42,...,40,50,26,7,3,22,26,109,17,1
654,22020,1610612738,BOS,Boston Celtics,0022000350,2021-02-05,BOS @ LAC,W,240,43,...,25,34,21,8,1,7,16,119,4,1


In [53]:
from nba_api.stats.endpoints import teamgamelog

teamgamelog.TeamGameLog(
    team_id="1610612738",
    season = 2020,
    date_from_nullable = '2020-11-17',
)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## NBA Schedule
This section creates logic to return the NBA schedule. The returned JSON will be filtered by
start and end date in order to return games scheduled in a desired Yahoo Fantasy week.

In [60]:
import requests
r = requests.get("http://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2020/league/00_full_schedule.json")

In [61]:
r.json()

{'lscd': [{'mscd': {'mon': 'December',
    'g': [{'gid': '0012000001',
      'gcode': '20201211/ORLATL',
      'seri': '',
      'is': 1,
      'gdte': '2020-12-11',
      'htm': '2020-12-11T19:00:00',
      'vtm': '2020-12-11T19:00:00',
      'etm': '2020-12-11T19:00:00',
      'an': 'State Farm Arena',
      'ac': 'Atlanta',
      'as': 'GA',
      'st': '3',
      'stt': 'Final',
      'bd': {'b': [{'seq': 1,
         'disp': 'Fox Sports Southeast - Atlanta',
         'scope': 'home',
         'type': 'tv',
         'lan': 'English'},
        {'seq': 2,
         'disp': 'TSN',
         'scope': 'can',
         'type': 'tv',
         'lan': 'English'},
        {'seq': 3,
         'disp': 'WZGC 92.9 FM The Game',
         'scope': 'home',
         'type': 'radio',
         'lan': 'English'},
        {'seq': 4,
         'disp': 'WYGM 96.9FM / 740AM',
         'scope': 'away',
         'type': 'radio',
         'lan': 'English'}]},
      'v': {'tid': 1610612753,
       're': '1-0',
    